In [ ]:
# Assertion model load from directory and evaluate the model. Input of the model will be the output of NER model.
# NER model extracts the entity from sentence. Enclose the entity with [entity] tag. 

In [6]:
!pip install seqeval
!pip install transformers

You should consider upgrading via the '/Users/sajjadislam/opt/anaconda3/envs/py_venv_nmdsi/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/sajjadislam/opt/anaconda3/envs/py_venv_nmdsi/bin/python -m pip install --upgrade pip' command.


In [7]:
import os
import pandas as pd
import math
import numpy as np
from tqdm import tqdm, trange
from seqeval.metrics import classification_report, accuracy_score, f1_score
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoModel, AutoConfig, AutoTokenizer
from transformers import AdamW
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification

In [26]:
# no of classifier: present, not-present
num_labels = 2

In [38]:
# no of classifier: present, possible, not-present
num_labels = 3

In [63]:
# no of classifier: present, possible, conditional, not-present
num_labels = 4

In [12]:
# no of classifier: present, possible, conditional, associated_with_someone_else, not-present
num_labels = 5

In [3]:
# no of classifier: present, possible, conditional, associated_with_someone_else, hypothetical, absent
num_labels = 6

In [10]:
MODEL_CLASSES = {
  'bert': (AutoConfig, BertForSequenceClassification, AutoTokenizer),
}
MODEL_ADDRESS = 'emilyalsentzer/Bio_ClinicalBERT'
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
model_config = config_class.from_pretrained(MODEL_ADDRESS, num_labels=num_labels)
tokenizer = tokenizer_class.from_pretrained(MODEL_ADDRESS, do_lower_case=False)
model = model_class.from_pretrained(MODEL_ADDRESS, config=model_config)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [28]:
output_dir = './trained_models/2_label_model/'
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

# Copy the model to the GPU.
# model.to(device)

In [40]:
output_dir = './trained_models/3_label_model/'
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

In [65]:
output_dir = './trained_models/4_label_model/'
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

In [14]:
output_dir = './trained_models/5_label_model/'
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

In [15]:
output_dir = './trained_models/6_label_model/'
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

In [4]:
output_dir = './trained_models/6_label_model_weight_balance/'
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

In [5]:
output_dir = './trained_models/6_label_model_oversampling/'
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

**Predict with model**

In [ ]:
sentences = ['Patient has [entity] fever [entity].', 'Patient denies [entity] fever [entity].']

sentences = ['Patient has fever.', 'Patient denies fever.']

sentences = ['There was an initial murmur on admission likely secondary to [entity] severe anemia [entity] which has since resolved .']

sentences = ['He denies a history of [entity] aspiration events [entity] or choking with food intake .']

In [ ]:
sentences = ['She had [entity] a functioning arteriovenous fistula [entity] with a thrill and a bruit in her left arm .', #present
    'This could be due to internal hernia or could be [entity] stricture [entity] related .', #possible
    'Her heart is regular without [entity] murmurs [entity] , rubs or gallops , slightly tachycardic .	', #absent
    'Call your doctor immediately if any new symptoms develop including fevers , [entity] rash [entity],'+
    'increase in bloody urine in nephrostomy or urostomy bags , etc .', #hypothetical
    'He has has [entity] stable , mild angina [entity] , subsequently , mostly in the setting of stress .', #conditional
    'She has a son who is [entity] mentally handicapped [entity] .' #associated_with_someone_else
    ]

print(type(sentences))

<class 'list'>


Read input from file 

In [6]:
xls = pd.ExcelFile('../Data/Test_ast_model_3_label.xlsx', engine='openpyxl')
df_test_present = pd.read_excel(xls, 'present')
df_test_absent = pd.read_excel(xls, 'absent')
df_test_possible = pd.read_excel(xls, 'possible')
df_test_conditional = pd.read_excel(xls, 'conditional')
df_test_hyphothetical = pd.read_excel(xls, 'hyphothetical')
df_test_associated = pd.read_excel(xls, 'associated with someone else')

present_list = df_test_present['Sentence'].tolist()
absent_list = df_test_absent['Sentence'].tolist()
possible_list = df_test_possible['Sentence'].tolist()
conditional_list = df_test_conditional['Sentence'].tolist()
hyphothetical_list = df_test_hyphothetical['Sentence'].tolist()
associated_list = df_test_associated['Sentence'].tolist()

sentences = conditional_list


In [9]:
xls = pd.ExcelFile('../Data/Test_ast_model_4_label.xlsx', engine='openpyxl')
df_test_present = pd.read_excel(xls, 'present')
df_test_absent = pd.read_excel(xls, 'absent')
df_test_possible = pd.read_excel(xls, 'possible')
df_test_conditional = pd.read_excel(xls, 'conditional')
df_test_hyphothetical = pd.read_excel(xls, 'hyphothetical')
df_test_associated = pd.read_excel(xls, 'associated with someone else')

present_list = df_test_present['Sentence'].tolist()
absent_list = df_test_absent['Sentence'].tolist()
possible_list = df_test_possible['Sentence'].tolist()
conditional_list = df_test_conditional['Sentence'].tolist()
hyphothetical_list = df_test_hyphothetical['Sentence'].tolist()
associated_list = df_test_associated['Sentence'].tolist()

sentences = associated_list

In [45]:
xls = pd.ExcelFile('../Data/Test_ast_model_5_label.xlsx', engine='openpyxl')
df_test_present = pd.read_excel(xls, 'present')
df_test_absent = pd.read_excel(xls, 'absent')
df_test_possible = pd.read_excel(xls, 'possible')
df_test_conditional = pd.read_excel(xls, 'conditional')
df_test_hyphothetical = pd.read_excel(xls, 'hyphothetical')
df_test_associated = pd.read_excel(xls, 'associated with someone else')

present_list = df_test_present['Sentence'].tolist()
absent_list = df_test_absent['Sentence'].tolist()
possible_list = df_test_possible['Sentence'].tolist()
conditional_list = df_test_conditional['Sentence'].tolist()
hyphothetical_list = df_test_hyphothetical['Sentence'].tolist()
associated_list = df_test_associated['Sentence'].tolist()

sentences = hyphothetical_list

In [6]:
xls = pd.ExcelFile('../Data/Test_ast_model_6_label.xlsx', engine='openpyxl')
df_test_present = pd.read_excel(xls, 'present')
df_test_absent = pd.read_excel(xls, 'absent')
df_test_possible = pd.read_excel(xls, 'possible')
df_test_conditional = pd.read_excel(xls, 'conditional')
df_test_hyphothetical = pd.read_excel(xls, 'hyphothetical')
df_test_associated = pd.read_excel(xls, 'associated with someone else')

present_list = df_test_present['Sentence'].tolist()
absent_list = df_test_absent['Sentence'].tolist()
possible_list = df_test_possible['Sentence'].tolist()
conditional_list = df_test_conditional['Sentence'].tolist()
hyphothetical_list = df_test_hyphothetical['Sentence'].tolist()
associated_list = df_test_associated['Sentence'].tolist()

sentences = conditional_list

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Test_ast_model_6_label.xlsx'

In [ ]:
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
      
    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
# labels = torch.tensor(labels)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/sajjadislam/opt/anaconda3/envs/py_venv_nmdsi/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

/var/folders/p3/s4n39zb50rb2l96w9n054ch80000gn/T/ipykernel_6619/2464600030.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids)
/var/folders/p3/s4n39zb50rb2l96w9n054ch80000gn/T/ipykernel_6619/2464600030.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks = torch.tensor(attention_masks)


In [9]:
predictions = []

In [10]:
model.eval()

with torch.no_grad():
    result = model(input_ids, token_type_ids=None, attention_mask=attention_masks, return_dict=True)

logits = result.logits
logits = logits.detach().cpu().numpy()
predictions.append(logits)

# print('sentences: ', sentences)
pred_labels_i = np.argmax(logits, axis=1).flatten()
# print('Label prediction: ', pred_labels_i) 


In [ ]:
for index, sentence in enumerate(sentences):
  print(sentence)
  print(pred_labels_i[index])
  if pred_labels_i[index] == 0:
    print ('Present')
  elif pred_labels_i[index] == 1:
    print ('Not-present')


In [ ]:
file = open('Test_output.txt', 'w')
for index, sentence in enumerate(sentences):
  print(sentence)
  file.write(sentence)
  file.write('\n')
  if pred_labels_i[index] == 0:
    print ('Present')
    file.write('Present')
  elif pred_labels_i[index] == 1:
    print ('Possible')
    file.write('Possible')
  elif pred_labels_i[index] == 2:
    print ('Not-present')
    file.write('Not-present')
  file.write('\n')
  
file.close()

In [ ]:

for index, sentence in enumerate(sentences):
  print(sentence)
  if pred_labels_i[index] == 0:
    print ('Present')
  elif pred_labels_i[index] == 1:
    print ('Possible')
  elif pred_labels_i[index] == 2:
    print ('Conditional')
  elif pred_labels_i[index] == 3:
    print ('Not-present')
  


In [ ]:
for index, sentence in enumerate(sentences):
  print(sentence)
  if pred_labels_i[index] == 0:
    print ('Present')
  elif pred_labels_i[index] == 1:
    print ('Possible')
  elif pred_labels_i[index] == 2:
    print ('Conditional')
  elif pred_labels_i[index] == 3:
    print ('associated with someone else')
  elif pred_labels_i[index] == 4:
    print ('Not-present')

In [11]:
for index, sentence in enumerate(sentences):
  print(sentence)
  if pred_labels_i[index] == 0:
    print ('Present')
  elif pred_labels_i[index] == 1:
    print ('Possible')
  elif pred_labels_i[index] == 2:
    print ('Conditional')
  elif pred_labels_i[index] == 3:
    print ('associated with someone else')
  elif pred_labels_i[index] == 4:
    print ('hypothetical')
  elif pred_labels_i[index] == 5:
    print ('Absent')

She had [entity] a functioning arteriovenous fistula [entity] with a thrill and a bruit in her left arm .
Present
This could be due to internal hernia or could be [entity] stricture [entity] related .
Possible
Her heart is regular without [entity] murmurs [entity] , rubs or gallops , slightly tachycardic .	
Absent
Call your doctor immediately if any new symptoms develop including fevers , [entity] rash [entity],increase in bloody urine in nephrostomy or urostomy bags , etc .
hypothetical
He has has [entity] stable , mild angina [entity] , subsequently , mostly in the setting of stress .
Present
She has a son who is [entity] mentally handicapped [entity] .
associated with someone else
